In [ ]:
### FILTRA OS PRATOS EXTRAIDOS DO ALLRECIPES, SELECIONANDO APENAS OS QUE TEM INGREDIENTES 
### PRESENTES NA BASE DE INGREDIENTES (COM SABOR ASSOCIADO) ###


In [16]:
import pandas as pd
from fuzzywuzzy import fuzz, process


In [20]:
# Carregar as bases de dados
receitas_df = pd.read_csv('./datasets/raw/ALLRECIPES_PRATOSxINGREDIENTES.csv')
ingredientes_df = pd.read_csv('./datasets/preprocessed/INGREDIENTES_preprocessed.csv')

In [22]:
ingredientes_df

,ingrediente,categoria,peso_sabores
0,bakery products,grain,"Counter({'nutty': 5, 'peanut': 5, 'cocoa': 5, ..."
1,bread,grain,"Counter({'sweet': 39, 'bitter': 35, 'fruity': ..."
2,rye bread,grain,"Counter({'sweet': 14, 'fruity': 8, 'bitter': 6..."
3,wheaten bread,grain,"Counter({'sweet': 10, 'fruity': 8, 'green': 7,..."
4,soda farls,grain,"Counter({'sweet': 10, 'fruity': 8, 'green': 7,..."
...,...,...,...
1934,guinea hen,protein,"Counter({'very mild': 1, 'grassy': 1, 'bitter'..."
1935,original fowl,protein,"Counter({'very mild': 1, 'grassy': 1, 'bitter'..."
1936,pet speckled hen,protein,"Counter({'very mild': 1, 'grassy': 1, 'bitter'..."
1937,guinea fowl,protein,"Counter({'very mild': 1, 'grassy': 1, 'bitter'..."


In [24]:
# FILTRA APENAS AS RECEITAS QUE CONTEM INGREDIENTES PRESENTE NA BASE DE INGREDIENTES
# LEVANDO EM CONTA PALAVRAS SEMELHANTES USANDO O FUZZYWUZZY

# CRIANDO UMA COLUNA A MAIS DE INGREDIENTES CONTENDO OS INGREDIENTES SIMILARES ENCONTRADOS NA BASE DE INGREDIENTE
# ESSA NOVA COLUNA É A QUE VAI FICAR VALENDO

# Converter as colunas de ingredientes em conjunto (se for string)
receitas_df['ingredientes'] = receitas_df['ingredientes'].apply(eval)  # Caso os ingredientes estejam no formato string

# Criar um conjunto de ingredientes da base de ingredientes
ingredientes_set = set(ingredientes_df['ingrediente'])

# Função para normalizar ingredientes usando fuzzywuzzy
def normalizar_ingredientes(ingredientes):
    ingredientes_normalizados = set()
    for ingrediente in ingredientes:
        # Usar fuzzywuzzy para encontrar a melhor correspondência
        match = process.extractOne(ingrediente, ingredientes_set)
        if match and match[1] >= 80:  # Pontuação mínima de 80 para similaridade
            ingredientes_normalizados.add(match[0])
    return ingredientes_normalizados

# Função para filtrar receitas que têm todos os ingredientes encontrados
def filtrar_receitas(row):
    ingredientes = row['ingredientes']
    ingredientes_normalizados = normalizar_ingredientes(ingredientes)
    
    # Verificar se o número de ingredientes normalizados é igual ao número de ingredientes originais
    if len(ingredientes_normalizados) == len(ingredientes):
        return ingredientes_normalizados  # Mantém a receita se todos os ingredientes tiverem correspondência
    else:
        return None  # Descartar a receita se não todos os ingredientes forem encontrados

# Aplicar o filtro nas receitas
receitas_df['ingredientes_normalizados'] = receitas_df.apply(filtrar_receitas, axis=1)

# Remover receitas onde a normalização resultou em None (ingredientes não encontrados)
receitas_filtradas = receitas_df[receitas_df['ingredientes_normalizados'].notna()]

# Remover quaisquer linhas com NaN na coluna 'prato'
receitas_filtradas = receitas_filtradas[receitas_filtradas['prato'].notna()]

# Exibir o resultado final
print(receitas_filtradas)


                                            prato  \
2                              air fryer eggplant   
3                      shiitake mushroom stir fry   
28             eggplant caponata sicilian version   
29                              caprese appetizer   
36                      caramel apple pie cookies   
...                                           ...   
1580            easy pumpkin spice coffee creamer   
1581     sausage and cabbage stuffed acorn squash   
1582                                 candied yams   
1584                                 roasted yams   
1587  hot honey butter cream cheese stuffed bagel   

                                           ingredientes  \
2     {salt, lemon pepper, thyme, eggplant, avocado ...   
3     {onion, soy sauce, orange pepper, zucchini, sh...   
28    {onion, garlic, parsley, salt, capers, plum to...   
29    {salt, tomatoes, black pepper, basil leaves, m...   
36     {sugar, apple pie, half and half, caramel sauce}   
...      

In [106]:
receitas_filtradas = pd.read_csv('./datasets/raw/ALLRECIPES_PRATOS_filtrado.csv')

In [108]:
print(receitas_filtradas)

                                           prato  \
0                             air fryer eggplant   
1                     shiitake mushroom stir fry   
2             eggplant caponata sicilian version   
3                              caprese appetizer   
4                      caramel apple pie cookies   
..                                           ...   
408                        baked pumpkin polenta   
409            easy pumpkin spice coffee creamer   
410                                 candied yams   
411                                 roasted yams   
412  hot honey butter cream cheese stuffed bagel   

                                          ingredientes  \
0    {'eggplant', 'garlic powder', 'salt', 'thyme',...   
1    {'onion', 'cloves', 'soy sauce', 'zucchini', '...   
2    {'onion', 'sugar', 'green olives', 'celery', '...   
3    {'salt', 'basil leaves', 'mozzarella cheese', ...   
4    {'apple pie', 'sugar', 'caramel sauce', 'half ...   
..                         

In [112]:
# Remover as linhas onde a coluna 'ingredientes' ou 'ingredientes_normalizados' contém conjuntos vazios
receitas_filtradas = receitas_filtradas[
    (receitas_filtradas['ingredientes'].apply(lambda x: len(x) > 0)) & 
    (receitas_filtradas['ingredientes_normalizados'].apply(lambda x: len(x) > 0))
]

# Exibir o resultado após a filtragem
print(receitas_filtradas)


                                           prato  \
0                             air fryer eggplant   
1                     shiitake mushroom stir fry   
2             eggplant caponata sicilian version   
3                              caprese appetizer   
4                      caramel apple pie cookies   
..                                           ...   
408                        baked pumpkin polenta   
409            easy pumpkin spice coffee creamer   
410                                 candied yams   
411                                 roasted yams   
412  hot honey butter cream cheese stuffed bagel   

                                          ingredientes  \
0    {'eggplant', 'garlic powder', 'salt', 'thyme',...   
1    {'onion', 'cloves', 'soy sauce', 'zucchini', '...   
2    {'onion', 'sugar', 'green olives', 'celery', '...   
3    {'salt', 'basil leaves', 'mozzarella cheese', ...   
4    {'apple pie', 'sugar', 'caramel sauce', 'half ...   
..                         

In [114]:
# Deletar a antiga coluna 'ingredientes'
receitas_filtradas = receitas_filtradas.drop(columns=['ingredientes'])

# Renomear a coluna 'ingredientes_normalizados' para 'ingredientes'
receitas_filtradas = receitas_filtradas.rename(columns={'ingredientes_normalizados': 'ingredientes'})

# Exibir o DataFrame após as alterações
print(receitas_filtradas)


                                           prato  \
0                             air fryer eggplant   
1                     shiitake mushroom stir fry   
2             eggplant caponata sicilian version   
3                              caprese appetizer   
4                      caramel apple pie cookies   
..                                           ...   
408                        baked pumpkin polenta   
409            easy pumpkin spice coffee creamer   
410                                 candied yams   
411                                 roasted yams   
412  hot honey butter cream cheese stuffed bagel   

                                          ingredientes  
0    {'pepper', 'thyme', 'eggplant', 'garlic', 'avo...  
1    {'onion', 'soya sauce', 'pepper', 'mushroom', ...  
2    {'onion', 'plum', 'sugar', 'celery', 'olive', ...  
3    {'pepper', 'olive', 'tomato', 'salt', 'mozzare...  
4    {'apple', 'frog fish and sea devils', 'sugar',...  
..                               

In [116]:

# Salvar a nova base de dados em um arquivo CSV
receitas_filtradas.to_csv('./datasets/raw/ALLRECIPES_PRATOS_filtrado.csv', index=False)